# SAFENY Analysis Notebook
## Exploratory Data Analysis & Key Metrics for Judges

This notebook loads cleaned data from DuckDB and generates the 5 core metrics that tell the SAFENY story:
1. **Hottest Intersections** - Where enforcement should focus
2. **Temporal Patterns** - When violations peak (hour, day, season)
3. **Repeat Offenders** - Habitual speeding violators
4. **Risk by Borough** - Geographic enforcement prioritization
5. **Speed Violation Trends** - Understanding the severity landscape

## Section 1: Environment Setup & Path Configuration

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import duckdb

# Configure paths from notebooks/ to parent directories
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
DUCKDB_PATH = DATA_DIR / "duckdb" / "test.duckdb"
OPENDATA_DIR = DATA_DIR / "opendata"
CLEANED_DIR = OPENDATA_DIR / "cleaned"
SRC_DIR = PROJECT_ROOT / "src"

# Add src to path for imports
sys.path.insert(0, str(SRC_DIR))

print("✅ Environment Configuration:")
print(f"  Project Root: {PROJECT_ROOT}")
print(f"  Data Directory: {DATA_DIR}")
print(f"  DuckDB Path: {DUCKDB_PATH}")
print(f"  Cleaned Data: {CLEANED_DIR}")
print(f"  Source Code: {SRC_DIR}")

# Verify pyarrow
try:
    import pyarrow
    print(f"  PyArrow: {pyarrow.__version__} ✓")
except ImportError:
    print("  ❌ PyArrow not installed!")
    
# Verify DuckDB
try:
    print(f"  DuckDB: {duckdb.__version__} ✓")
except:
    print(f"  DuckDB: installed ✓")


## Section 2: Connect to DuckDB & Verify Data

In [ ]:
# Connect to DuckDB
conn = duckdb.connect(str(DUCKDB_PATH), read_only=True)
print(f"🔌 Connected to: {DUCKDB_PATH}")

# Check available tables
tables = conn.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema='main'
    ORDER BY table_name
""").fetchdf()

print(f"\n📊 Available Tables ({len(tables)}):")
print(tables.to_string())

# Quick stats
try:
    stats = {
        'Total Violations': conn.execute("SELECT COUNT(*) FROM fct_violations").fetchone()[0],
        'Unique Drivers': conn.execute("SELECT COUNT(DISTINCT driver_id) FROM fct_violations").fetchone()[0],
        'Unique Locations': conn.execute("SELECT COUNT(DISTINCT county) FROM fct_violations").fetchone()[0],
        'Speed Cameras': conn.execute("SELECT COUNT(*) FROM fct_violations WHERE data_source='SPEED_CAMERA'").fetchone()[0],
        'Traffic Violations': conn.execute("SELECT COUNT(*) FROM fct_violations WHERE data_source='TRAFFIC_VIOLATIONS'").fetchone()[0],
    }
    print("\n📈 Data Volume:")
    for key, val in stats.items():
        print(f"  {key}: {val:,}")
except Exception as e:
    print(f"⚠️ Data not yet loaded. Run: python src/ingestion.py first")
    print(f"   Error: {e}")


## Section 3: Metric 1 - HOTTEST INTERSECTIONS
### Where should enforcement focus? Top locations by violation concentration

In [ ]:
try:
    hottest_locations = conn.execute("""
        SELECT 
            county,
            COUNT(*) as total_violations,
            COUNT(CASE WHEN violation_date >= TODAY() - INTERVAL 30 DAY THEN 1 END) as violations_last_30_days,
            ROUND(AVG(COALESCE(points_assessed, 0)), 2) as avg_points,
            ROUND(AVG(COALESCE(fine_amount, 0)), 2) as avg_fine,
            COUNT(DISTINCT driver_id) as unique_drivers
        FROM fct_violations
        GROUP BY county
        ORDER BY total_violations DESC
        LIMIT 15
    """).fetchdf()
    
    print("🔥 TOP 15 LOCATIONS BY VIOLATION COUNT:")
    print(hottest_locations.to_string(index=False))
    print(f"\n✅ Total {len(hottest_locations)} counties with violations")
    
except Exception as e:
    print(f"❌ Query failed: {e}")


## Section 4: Metric 2 - TEMPORAL PATTERNS
### When do violations peak? Hourly, daily, and seasonal trends

In [ ]:
try:
    # Monthly trends
    monthly_trends = conn.execute("""
        SELECT 
            violation_year,
            violation_month,
            COUNT(*) as violation_count,
            ROUND(AVG(COALESCE(points_assessed, 0)), 2) as avg_points,
            COUNT(CASE WHEN data_source='SPEED_CAMERA' THEN 1 END) as speed_camera_violations
        FROM fct_violations
        WHERE violation_year IS NOT NULL AND violation_month IS NOT NULL
        GROUP BY violation_year, violation_month
        ORDER BY violation_year DESC, violation_month DESC
        LIMIT 12
    """).fetchdf()
    
    print("📅 RECENT MONTHLY TRENDS (Last 12 Months):")
    print(monthly_trends.to_string(index=False))
    
    # Day of week patterns
    dow_trends = conn.execute("""
        SELECT 
            violation_day_of_week,
            COUNT(*) as violation_count,
            ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM fct_violations WHERE violation_day_of_week IS NOT NULL), 1) as pct_of_total,
            ROUND(AVG(COALESCE(points_assessed, 0)), 2) as avg_points
        FROM fct_violations
        WHERE violation_day_of_week IS NOT NULL
        GROUP BY violation_day_of_week
        ORDER BY violation_count DESC
    """).fetchdf()
    
    print("\n📊 VIOLATIONS BY DAY OF WEEK:")
    print(dow_trends.to_string(index=False))
    
except Exception as e:
    print(f"❌ Query failed: {e}")


## Section 5: Metric 3 - REPEAT OFFENDERS
### Who are the habitual speeders? Top violators by violation count

In [ ]:
try:
    repeat_offenders = conn.execute("""
        SELECT 
            driver_id,
            COUNT(*) as violation_count,
            SUM(COALESCE(points_assessed, 0)) as total_points,
            ROUND(AVG(COALESCE(driver_age, 0)), 0) as avg_age,
            MIN(violation_date) as first_violation,
            MAX(violation_date) as last_violation,
            COUNT(CASE WHEN violation_date >= TODAY() - INTERVAL 1 YEAR THEN 1 END) as violations_last_year,
            CASE 
                WHEN COUNT(*) >= 10 THEN 'CRITICAL'
                WHEN COUNT(*) >= 5 THEN 'HIGH'
                ELSE 'MEDIUM'
            END as risk_tier
        FROM fct_violations
        WHERE driver_id IS NOT NULL
        GROUP BY driver_id
        HAVING COUNT(*) >= 2
        ORDER BY violation_count DESC
        LIMIT 20
    """).fetchdf()
    
    print("👤 TOP 20 REPEAT OFFENDERS:")
    print(repeat_offenders.to_string(index=False))
    
    # Tier summary
    tier_summary = conn.execute("""
        SELECT 
            CASE 
                WHEN COUNT(*) >= 10 THEN 'CRITICAL'
                WHEN COUNT(*) >= 5 THEN 'HIGH'
                ELSE 'MEDIUM'
            END as risk_tier,
            COUNT(DISTINCT driver_id) as driver_count,
            COUNT(*) as total_violations
        FROM fct_violations
        WHERE driver_id IS NOT NULL
        GROUP BY risk_tier
        ORDER BY driver_count DESC
    """).fetchdf()
    
    print("\n📊 REPEAT OFFENDER TIER SUMMARY:")
    print(tier_summary.to_string(index=False))
    
except Exception as e:
    print(f"❌ Query failed: {e}")


## Section 6: Metric 4 - VIOLATION BREAKDOWN
### What types of violations dominate? Speed vs. traffic infractions

In [ ]:
try:
    # By data source
    by_source = conn.execute("""
        SELECT 
            data_source,
            COUNT(*) as violation_count,
            ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM fct_violations), 1) as pct_of_total,
            ROUND(AVG(COALESCE(points_assessed, 0)), 2) as avg_points,
            ROUND(AVG(COALESCE(fine_amount, 0)), 2) as avg_fine,
            COUNT(DISTINCT driver_id) as unique_drivers
        FROM fct_violations
        GROUP BY data_source
        ORDER BY violation_count DESC
    """).fetchdf()
    
    print("📋 VIOLATIONS BY SOURCE:")
    print(by_source.to_string(index=False))
    
    # Top violation codes
    top_codes = conn.execute("""
        SELECT 
            violation_code,
            COUNT(*) as violation_count,
            ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM fct_violations), 1) as pct_of_total,
            ROUND(AVG(COALESCE(points_assessed, 0)), 2) as avg_points,
            COUNT(DISTINCT driver_id) as unique_drivers
        FROM fct_violations
        WHERE violation_code IS NOT NULL
        GROUP BY violation_code
        ORDER BY violation_count DESC
        LIMIT 15
    """).fetchdf()
    
    print("\n🚗 TOP 15 VIOLATION CODES:")
    print(top_codes.to_string(index=False))
    
except Exception as e:
    print(f"❌ Query failed: {e}")


## Section 7: Metric 5 - DEMOGRAPHIC PATTERNS
### Age distribution and violation patterns by driver age

In [ ]:
try:
    age_breakdown = conn.execute("""
        SELECT 
            CASE 
                WHEN driver_age < 25 THEN '16-24'
                WHEN driver_age < 35 THEN '25-34'
                WHEN driver_age < 45 THEN '35-44'
                WHEN driver_age < 55 THEN '45-54'
                WHEN driver_age < 65 THEN '55-64'
                ELSE '65+'
            END as age_group,
            COUNT(*) as violation_count,
            ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM fct_violations WHERE driver_age IS NOT NULL), 1) as pct_of_total,
            COUNT(DISTINCT driver_id) as unique_drivers,
            ROUND(AVG(COALESCE(points_assessed, 0)), 2) as avg_points,
            ROUND(AVG(COALESCE(driver_age, 0)), 1) as avg_age_in_group
        FROM fct_violations
        WHERE driver_age IS NOT NULL
        GROUP BY age_group
        ORDER BY MIN(driver_age)
    """).fetchdf()
    
    print("👥 VIOLATIONS BY AGE GROUP:")
    print(age_breakdown.to_string(index=False))
    
    # Age stats
    age_stats = conn.execute("""
        SELECT 
            ROUND(AVG(driver_age), 1) as avg_driver_age,
            MIN(driver_age) as min_age,
            MAX(driver_age) as max_age,
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY driver_age) as median_age
        FROM fct_violations
        WHERE driver_age IS NOT NULL
    """).fetchdf()
    
    print("\n📊 AGE STATISTICS:")
    print(age_stats.to_string(index=False))
    
except Exception as e:
    print(f"❌ Query failed: {e}")
